# Clustering Crypto

In [1]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
#import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from os import path

In [2]:
!pip install -U altair

In [3]:
import altair as alt

### Fetching Cryptocurrency Data

In [4]:
# Alternatively, use the provided csv file:
# Create a DataFrame
crypto_df = pd.read_csv("crypto_data.csv")
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [5]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1247,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [6]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [7]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != "N/A"]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [8]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns=["IsTrading"])
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
1243,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,SHA-256,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [9]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [10]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df["TotalCoinsMined"] != 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [11]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.loc[(crypto_df["CoinName"] != "N/A") & (crypto_df["Algorithm"] != "N/A") 
                            & (crypto_df["ProofType"] != "N/A") & (crypto_df["TotalCoinsMined"] != "N/A") & (crypto_df["TotalCoinSupply"] != "N/A")]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [12]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df["CoinName"]
CoinName_df

0           42 Coin
2           404Coin
5         EliteCoin
7           Bitcoin
8          Ethereum
           ...     
1238         ZEPHYR
1242        Gapcoin
1245         Beldex
1246        Horizen
1247    BitcoinPlus
Name: CoinName, Length: 533, dtype: object

In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns=["CoinName"])
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [14]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=["Algorithm","ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1242,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Standardize data
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [16]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
x_info = pca.fit_transform(X_scaled)
x_info

array([[-0.32969643,  1.08163018, -0.51925394],
       [-0.31301265,  1.08136298, -0.51999402],
       [ 2.306906  ,  1.56699353, -0.72384148],
       ...,
       [ 0.32162588, -2.35838983,  0.30334036],
       [-0.15773725, -1.96175437,  0.40059946],
       [-0.28388362,  0.87711884, -0.21181229]])

In [17]:
pca.explained_variance_ratio_

array([0.02737269, 0.02091903, 0.02008526])

In [18]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(x_info, columns=["PC1","PC2","PC3"])
#pcs_df.set_index(CoinName_df["CoinName"])
pcs_df.set_index(CoinName_df)

,PC1,PC2,PC3
CoinName,,,
42 Coin,-0.329696,1.081630,-0.519254
404Coin,-0.313013,1.081363,-0.519994
EliteCoin,2.306906,1.566994,-0.723841
Bitcoin,-0.147213,-1.352304,0.143433
Ethereum,-0.152615,-2.063841,0.341654
...,...,...,...
ZEPHYR,2.470050,1.023228,-0.125320
Gapcoin,-0.327740,1.081458,-0.519317
Beldex,0.321626,-2.358390,0.303340


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [19]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(pcs_df)
    inertia.append(k_model.inertia_)

# Create the Elbow Curve using altair
elbow_data = {"k":k, "inertia":inertia}
elbow_df = pd.DataFrame(elbow_data)
print(elbow_df)
alt.Chart(elbow_df).mark_line().encode(x="k",y="inertia")

    k      inertia
0   1  3644.493100
1   2  2474.956982
2   3  1489.386033
3   4   597.176411
4   5   444.368627
5   6   281.633696
6   7   215.038421
7   8   156.672401
8   9   129.956024
9  10   105.280069


alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [20]:
# Initialize the K-Means model
k_model = KMeans(n_clusters=4, random_state=1)

# Fit the model
k_model.fit(pcs_df)

# Predict clusters
k_4 = k_model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_df = crypto_df.reset_index(drop=True)
pcs_df = pcs_df.reset_index(drop=True)
CoinName_df = CoinName_df.reset_index(drop=True)
clustered_df = crypto_df.copy()
clustered_df = pd.concat([clustered_df, pcs_df],axis=1)
clustered_df["CoinName"] = CoinName_df
clustered_df["Class"] = k_4
clustered_df = clustered_df.set_index(CoinName_df)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
CoinName,,,,,,,,,
42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.329696,1.081630,-0.519254,42 Coin,0
404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.313013,1.081363,-0.519994,404Coin,0
EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.306906,1.566994,-0.723841,EliteCoin,0
Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.147213,-1.352304,0.143433,Bitcoin,1
Ethereum,Ethash,PoW,1.076842e+08,0,-0.152615,-2.063841,0.341654,Ethereum,1
...,...,...,...,...,...,...,...,...,...
ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000,2.470050,1.023228,-0.125320,ZEPHYR,0
Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.327740,1.081458,-0.519317,Gapcoin,0
Beldex,CryptoNight,PoW,9.802226e+08,1400222610,0.321626,-2.358390,0.303340,Beldex,1


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [21]:
# Plot the scatter with x="PC1" and y="PC2"
alt.Chart(clustered_df).mark_point().encode(
    x="PC1",
    y="PC2",
    color="Class",
    tooltip=["CoinName","Algorithm","TotalCoinSupply","TotalCoinsMined"]
)

alt.Chart(...)

In [22]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()
df_columns = ["TotalCoinsMined", "TotalCoinSupply"]

X = clustered_df[df_columns].values
X_scaled = scaler.fit_transform(X)

tradable_crypto = pd.DataFrame(X_scaled, columns=df_columns, index=clustered_df.index).reset_index()
tradable_crypto

,CoinName,TotalCoinsMined,TotalCoinSupply
0,42 Coin,0.005942,4.200000e-11
1,404Coin,0.007002,5.320000e-04
2,EliteCoin,0.035342,3.141593e-01
3,Bitcoin,0.005960,2.100000e-05
4,Ethereum,0.006050,0.000000e+00
...,...,...,...
528,ZEPHYR,0.007951,2.000000e-03
529,Gapcoin,0.005957,2.500000e-04
530,Beldex,0.006927,1.400223e-03
531,Horizen,0.005950,2.100000e-05


In [23]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(tradable_crypto).mark_point().encode(
    x="TotalCoinsMined",
    y="TotalCoinSupply"
)

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [24]:
# Table with tradable cryptos
#clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"])
tradable_cryptocurrencies = clustered_df.drop(columns=["PC1","PC2","PC3"])
display(tradable_cryptocurrencies)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,Class
CoinName,,,,,,
42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,0
404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,0
EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,0
Bitcoin,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,1
Ethereum,Ethash,PoW,1.076842e+08,0,Ethereum,1
...,...,...,...,...,...,...
ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000,ZEPHYR,0
Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000,Gapcoin,0
Beldex,CryptoNight,PoW,9.802226e+08,1400222610,Beldex,1


In [25]:
# Print the total number of tradable cryptocurrencies
print(f"The total number of tradable cryptocurrencies: {len(tradable_cryptocurrencies)}")

The total number of tradable cryptocurrencies: 533
